## Mask R-CNN: freeze the graph

In [10]:
"""
Based on the work of Waleed Abdulla (Matterport)
Modified by github.com/GustavZ
"""

import os
import sys
import numpy as np
import tensorflow as tf


# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

# Model Directory 
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
WEIGHTS_DIR = os.path.join(ROOT_DIR, "weights")
DEFAULT_WEIGHTS_PATH = os.path.join(WEIGHTS_DIR, "ResNet-101", "mask_rcnn_coco.h5")

## Configurations

In [2]:
# MS COCO Dataset
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples", "coco"))  # To find local version
import coco
config = coco.CocoConfig()

Using TensorFlow backend.


In [3]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

## Notebook Preferences

In [4]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
#DEVICE = "/cpu:0"
DEVICE = "/gpu:0"

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"
#TEST_MODE = "training"

In [5]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Load Model

In [11]:
import mrcnn.model as modellib

# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode=TEST_MODE, model_dir=MODEL_DIR, config=config)

# Set path to model weights
weights_path = DEFAULT_WEIGHTS_PATH
#weights_path = model.find_last()[1]

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

Loading weights  /home/processyuan/code/HumanMask/my-Mobile-Mask-RCNN/weights/mask_rcnn_coco.h5


## Prepare and Export Model

In [12]:
# name of the pb file we want to output
MODEL_NAME = 'mask_rcnn_coco' # TODO: enter value here

# Chose whether to quantize the graph
QUANTIZE = False

In [13]:
from tensorflow.tools.graph_transforms import TransformGraph
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras import backend as K

# Get keras model and save
model_keras= model.keras_model
# All new operations will be in test mode from now on.
K.set_learning_phase(0)

# Create output layer with customized names
num_output = 7
pred_node_names = ["detections", "mrcnn_class", "mrcnn_bbox", "mrcnn_mask", "rois", "rpn_class", "rpn_bbox"]
pred_node_names = ["output_" + name for name in pred_node_names]
pred = [tf.identity(model_keras.outputs[i], name = pred_node_names[i])for i in range(num_output)]

# Get the object detection graph
sess = K.get_session()
if QUANTIZE:
    # Transformations
    transforms = ["quantize_weights", "quantize_nodes"]
    transformed_graph_def = TransformGraph(sess.graph.as_graph_def(), [], pred_node_names, transforms)
    constant_graph = graph_util.convert_variables_to_constants(sess, transformed_graph_def, pred_node_names)
    PB_MODEL_PATH = os.path.join(MODEL_DIR, MODEL_NAME+'_quantized'+'.pb') 
else:
    constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)
    PB_MODEL_PATH = os.path.join(MODEL_DIR, MODEL_NAME+'.pb') 

INFO:tensorflow:Froze 690 variables.
INFO:tensorflow:Converted 690 variables to const ops.


In [14]:
# Write Output pb File
graph_io.write_graph(constant_graph, "/", PB_MODEL_PATH, as_text=False)

# Output Info
print('{} ops in the frozen graph.'.format(len(constant_graph.node)))
print('saved the freezed graph (ready for inference) at: ', PB_MODEL_PATH)

2976 ops in the frozen graph.
saved the freezed graph (ready for inference) at:  /home/processyuan/code/HumanMask/my-Mobile-Mask-RCNN/logs/mask_rcnn_coco.pb
